In [ ]:
import couchdb
from pprint import pprint
from datetime import datetime

In [ ]:
user = "admin"
password = "password"
couchserver = couchdb.Server("http://%s:%s@couchdb:5984/" % (user, password))

In [ ]:
dbname = "mydb"
del couchserver[dbname]
if dbname in couchserver:
    db = couchserver[dbname]
else:
    db = couchserver.create(dbname)

In [ ]:
for dbname in couchserver:
    print(dbname)

In [ ]:
# datetime object containing current date and time
now = str(datetime.now().replace(second=0, microsecond=0))

docs = [   
    {'doc_type': 'category', 'name': 'users'},
    {'doc_type': 'category', 'name': 'packages'},
    
    {'doc_type': 'users', 'key': 'alice', 'name': 'Alice Appleworth', 'state': 'present', 'telephone': '123-456-7890'},
    {'doc_type': 'users', 'key': 'bob', 'name': 'Bob Bananarama', 'state': 'present', 'telephone': '987-654-3210'},
    {'doc_type': 'users', 'key': 'carlos', 'name': 'Carlos Carrot', 'state': 'absent', 'telephone': '555-123-6789'},
    
    {'doc_type': 'packages', 'name': 'executive_lounge.example.com', 'state': 'present', 'date': now},
    {'doc_type': 'pac_member', 'name': 'server1.example.com', 'state': 'present', 'parent': 'executive_lounge.example.com', 'date': now},
    {'doc_type': 'pac_member', 'name': 'server1.example.com', 'state': 'present', 'parent': 'executive_lounge.example.com', 'date': now},
    {'doc_type': 'packages', 'name': 'kitchen.example.com', 'state': 'present', 'date': now},
    {'doc_type': 'packages', 'name': 'bedroom.example.com', 'state': 'present', 'date': now}]

In [ ]:
for (success, doc_id, revision_or_exception) in db.update(docs):
    print(success, doc_id, revision_or_exception)

In [ ]:
rows = db.view('_all_docs', include_docs=True)
objects = [row['doc'] for row in rows if row['doc']['doc_type'] == 'category']
for o in objects:
    print(o['doc_type'] + " : " + o['name'])
    subobjects1 = [row['doc'] for row in rows if row['doc']['doc_type'] == o['name']]
    o[o['name']] = subobjects1
    for s1 in subobjects1:
        print("  -> " + s1['doc_type'] + " : " + s1['name'])
        subobjects2 = [row['doc'] for row in rows if 'parent' in row['doc']]
        subobjects2 = [i for i in subobjects2 if i['parent'] == s1['name']]
        if subobjects2:
            s1[s1['name']] = subobjects2
            for s2 in subobjects2:
                print("    -> " + s2['doc_type'] + " : " + s2['name'])

In [ ]:
pprint(objects)

In [ ]:
f = open("roles/demo/vars/main/couch_vars.yml", "w")
f.write("---" + '\n')
for obj in objects:
    var_name = obj['name']
    var_content = obj[var_name]
    f.write(var_name + ": " + str(var_content) + '\n' * 2)
f.close()

In [ ]:
%%bash
ls -larth ./roles/demo/vars/main

In [ ]:
%%bash
cat ./roles/demo/vars/main/couch_vars.yml